In [207]:
import pandas as pd
from numpy import nan
import re

In [233]:
unknown_protein_keywords = "unknown|Unknown|hypothetical|Hypothetical|Uncharacterized|Uncharacterised|uncharacterized|uncharacterised|DUF|unknow|putative protein in bacteria|Unassigned|unassigned"

## NOTE: some hypothetical proteins are "ISXX family insertion sequence hypothetical protein"
## so filter out those cases, when counting unknown proteins.


## match MGE genes using the following keywords in the "gene_product" annotation
IS_keywords = r"IS|transposon|Transposase|transposase|Transposable|transposable|virus|Phage|phage|integrase|Integrase|baseplate|tail|intron|Mobile|mobile|antitoxin|toxin|capsid|plasmid|Plasmid|conjug|Tra"

## Elongation Factor Tu (2 copies in most bacteria).
## \\b is a word boundary.
## see: https://stackoverflow.com/questions/62430498/detecting-whole-words-using-str-detect-in-r
EFTu_keywords = r"\\bTu | Tu\\b|-Tu\\b"

## antibiotic-specific keywords.
chloramphenicol_keywords = r"chloramphenicol|Chloramphenicol"
tetracycline_keywords = r"tetracycline|Tetracycline"
MLS_keywords = r"macrolide|lincosamide|streptogramin"
multidrug_keywords = r"multidrug"
beta_lactam_keywords = r"lactamase"
glycopeptide_keywords = r"glycopeptide resistance|VanZ"
polypeptide_keywords = r"bacitracin|polymyxin B"
diaminopyrimidine_keywords = r"trimethoprim-resistant"
sulfonamide_keywords = r"sulfonamide-resistant"
quinolone_keywords = r"quinolone|Quinolone|oxacin"
aminoglycoside_keywords = r"aminoglycoside|streptomycin|Streptomycin|kanamycin|Kanamycin|tobramycin|Tobramycin|gentamicin|Gentamicin|neomycin|Neomycin"
macrolide_keywords = r"macrolide|ketolide|Azithromycin|azithromycin|Clarithromycin|clarithromycin|Erythromycin|erythomycin"

antibiotic_keywords = r"chloramphenicol|Chloramphenicol|tetracycline|Tetracycline|macrolide|lincosamide|streptogramin|multidrug|lactamase|glycopeptide resistance|VanZ|bacitracin|polymyxin B|trimethoprim-resistant|sulfonamide-resistant|quinolone|Quinolone|oxacin|aminoglycoside|streptomycin|Streptomycin|kanamycin|Kanamycin|tobramycin|Tobramycin|gentamicin|Gentamicin|neomycin|Neomycin|macrolide|ketolide|Azithromycin|azithromycin|Clarithromycin|clarithromycin|Erythromycin|erythomycin|antibiotic resistance"
ARG_regex = re.compile(antibiotic_keywords)  

In [259]:
## get the samples in the antibiotic+ and antibiotic- cohorts.
DIABIMMUNE_samples_df = pd.read_csv("../data/DIABIMMUNE-antibiotics-cohort-data/DIABIMMUNE-SampleIDs.csv")
DIABIMMUNE_antibiotic_treatments_df = pd.read_csv("../data/DIABIMMUNE-antibiotics-cohort-data/DIABIMMUNE-AntibioticTreatments.csv")
all_samples = set([x for x in DIABIMMUNE_samples_df.Subject])
case_samples = set([x for x in DIABIMMUNE_antibiotic_treatments_df.Subject])
control_samples = all_samples - case_samples
## make a regex for filtering.
case_sample_string = "|".join(case_samples)
control_sample_string = "|".join(control_samples)
print(case_sample_string)
print(len(case_samples))
print()
print(control_sample_string)
print(len(control_samples))

E003953|E010481|E006781|E023445|E024907|E006091|E021940|E010682|E004628|E011878|E013505|E014403|E021822|E020924|E004898|E012854|E028794|E021235|E003188|E016426|E005786
21

E022497|E035134|E020570|E019763|E007944|E001958|E021032|E018754|E016063|E016273|E006493|E019092|E013094|E018286|E004709|E000823|E014086|E017497|E010581
19


In [167]:
## let's examine the provided dataframes for the reference genomes.
reference_taxonomy_df = pd.read_pickle("../data/SGVFinder/DataFiles/representatives.genomes.taxonomy.df")
## The annotations are in a multiindexed dataframe:
## https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.MultiIndex.html
## https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#hierarchical-indexing-multiindex
reference_annotations_df = pd.read_pickle("../data/SGVFinder/DataFiles/representatives.genes.drepped.annotations.df")
## rename the product column to protein_product since "product" is already in the pandas namespace...
reference_annotations_df = reference_annotations_df.rename(columns={'product': 'gene_product'})

In [3]:
reference_taxonomy_df

,Superkingdom,Phylum,Class,Order,Family,Genus,Species,organism
139,Bacteria,Spirochaetes,Spirochaetia,Spirochaetales,Borreliaceae,Borreliella,NaN,Borreliella burgdorferi
192,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodospirillales,Rhodospirillaceae,Azospirillum,NaN,Azospirillum brasilense
202,Bacteria,Proteobacteria,Epsilonproteobacteria,Campylobacterales,Campylobacteraceae,Campylobacter,NaN,Campylobacter mucosalis
210,Bacteria,Proteobacteria,Epsilonproteobacteria,Campylobacterales,Helicobacteraceae,Helicobacter,NaN,Helicobacter pylori
216,Bacteria,Proteobacteria,Epsilonproteobacteria,Campylobacterales,Helicobacteraceae,Helicobacter,NaN,Helicobacter muridarum
...,...,...,...,...,...,...,...,...
1565129,Bacteria,Proteobacteria,Gammaproteobacteria,Alteromonadales,Shewanellaceae,Shewanella,NaN,Shewanella sp. ECSMB14101
1565314,Bacteria,Proteobacteria,Epsilonproteobacteria,Campylobacterales,Campylobacteraceae,Sulfurospirillum,NaN,Sulfurospirillum sp. MES
1569209,Bacteria,Proteobacteria,Alphaproteobacteria,Rhodobacterales,Rhodobacteraceae,Paracoccus,NaN,Paracoccus sp. PAMC 22219
1577051,Bacteria,Deinococcus-Thermus,Deinococci,Thermales,Thermaceae,Thermus,NaN,Thermus sp. 2.9


In [168]:
reference_annotations_df

gene_type  start_pos    end_pos  \
dest               gene_name                                               
525903.PRJNA29531  Taci_0001                   CDS       96.0     1439.0   
                   Taci_0002                   CDS     1688.0     2827.0   
                   Taci_0003                   CDS     2817.0     3881.0   
                   Taci_0004                   CDS     3888.0     5756.0   
                   Taci_0005                   CDS     5753.0     6202.0   
...                                            ...        ...        ...   
244582.PRJNA255046 JQAK01000050_gene2511       CDS  2657906.0  2658316.0   
                   JQAK01000027_gene2512       CDS  2658410.0  2658565.0   
                   JQAK01000027_gene2513       CDS  2658660.0  2659103.0   
                   JQAK01000027_gene2514       CDS  2661236.0  2661781.0   
                   JQAK01000027_gene2515       CDS  2661783.0  2662049.0   

                                         strand  \
dest               gene_name                      
525903.PRJNA29531  Taci_0001                  +   
                   Taci_0002                  +   
                   Taci_0003                  +   
                   Taci_0004                  +   
                   Taci_0005                  +   
...                                         ...   
244582.PRJNA255046 JQAK01000050_gene2511      +   
                   JQAK01000027_gene2512      +   
                   JQAK01000027_gene2513      +   
                   JQAK01000027_gene2514      +   
                   JQAK01000027_gene2515      +   

                                                                            gene_product  
dest               gene_name                                                              
525903.PRJNA29531  Taci_0001              chromosomal replication initiator protein DnaA  
                   Taci_0002                            DNA polymerase III, beta subunit  
                   Taci_0003                     DNA replication and repair protein RecF  
                   Taci_0004                        glucose inhibited division protein A  
                   Taci_0005                          protein of unknown function DUF721  
...                                                                                  ...  
244582.PRJNA255046 JQAK01000050_gene2511                                                  
                   JQAK01000027_gene2512                                                  
                   JQAK01000027_gene2513                                                  
                   JQAK01000027_gene2514                                                  
                   JQAK01000027_gene2515                                                  

[13665098 rows x 5 columns]

In [235]:
ARG_annotations_idx = reference_annotations_df.gene_product.str.contains(ARG_regex, regex=True, na=False)

In [236]:
ARG_reference_annotations_df = reference_annotations_df[ARG_annotations_idx]

In [266]:
## rows are samples, and the columns are the reference genomes.
DIABIMMUNE_deletion_df = pd.read_pickle("../results/DIABIMMUNE-SGVFinder/dsgv.df")
DIABIMMUNE_structvar_df = pd.read_pickle("../results/DIABIMMUNE-SGVFinder/vsgv.df")

In [268]:
## let's restrict the samples to just the 23-24 month timepoint.
DIABIMMUNE_deletion_df = DIABIMMUNE_deletion_df[DIABIMMUNE_deletion_df.index.str.contains("_23|_24", regex=True, na=False)]
DIABIMMUNE_structvar_df = DIABIMMUNE_structvar_df[DIABIMMUNE_structvar_df.index.str.contains("_23|_24", regex=True, na=False)]

In [269]:
DIABIMMUNE_deletion_df

,216816.PRJNA251950:11_13;1172_1174;1174_1177,216816.PRJNA251950:28_29,216816.PRJNA251950:31_33;36_38,216816.PRJNA251950:56_59;132_133;259_261;262_264;264_265;723_727;1974_1983;2311_2313;2313_2317;2317_2318;2318_2321,216816.PRJNA251950:80_82;83_85;1887_1888;1888_1890,216816.PRJNA251950:91_92;1656_1657,216816.PRJNA251950:95_96;112_114;139_141;240_242;383_384;395_398;400_402;528_534;720_721;754_761;983_985;985_991;1022_1025;1038_1042;1108_1109;1200_1201;1315_1317;1336_1340;1429_1433;1605_1606;1824_1837;1944_1947;2132_2140;2182_2187;2189_2190;2229_2232;2309_2310;2321_2328;2370_2371,216816.PRJNA251950:146_147,216816.PRJNA251950:166_168,216816.PRJNA251950:192_193,...,657309.PRJNA39177:5303_5305;5305_5307,657309.PRJNA39177:5397_5398,657309.PRJNA39177:5595_5597,657309.PRJNA39177:5654_5656,657309.PRJNA39177:5679_5681,657309.PRJNA39177:5738_5739,657309.PRJNA39177:5891_5893;5893_5894,657309.PRJNA39177:5942_5944,657309.PRJNA39177:5944_5946,657309.PRJNA39177:5969_5970
E000823_24.3.periodic.trimmo.60.um.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E001958_24.1.periodic.trimmo.60.um.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E003188_24.2.periodic.trimmo.60.um.1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E003953_23.8.periodic.trimmo.60.um.1,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
E004628_24.5.periodic.trimmo.60.um.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E004709_24.7.periodic.trimmo.60.um.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E004898_23.8.periodic.trimmo.60.um.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0
E005786_23.9.periodic.trimmo.60.um.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E006091_23.5.periodic.trimmo.60.um.1,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E006493_24.1.periodic.trimmo.60.um.1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [270]:
DIABIMMUNE_structvar_df

,216816.PRJNA251950:1461_1462;1465_1466,216816.PRJNA251950:625_626,216816.PRJNA251950:1462_1465,216816.PRJNA251950:2361_2362,216816.PRJNA251950:2362_2363,216816.PRJNA251950:753_754;1845_1846,216816.PRJNA251950:56_57;528_529;530_531;982_991;1039_1041;1200_1201;1338_1339;1428_1433;1605_1606;1825_1826;1830_1831;1946_1947;2132_2134;2134_2140;2182_2189;2229_2230;2327_2328,216816.PRJNA251950:1974_1982,216816.PRJNA251950:757_759,216816.PRJNA251950:139_140,...,657309.PRJNA39177:638_641,657309.PRJNA39177:5087_5088,657309.PRJNA39177:4649_4650;4663_4665,657309.PRJNA39177:4648_4649,657309.PRJNA39177:4653_4663,657309.PRJNA39177:5027_5030;5033_5037;5040_5042;5045_5046,657309.PRJNA39177:5019_5021,657309.PRJNA39177:5021_5027,657309.PRJNA39177:5002_5008;5015_5018,657309.PRJNA39177:5009_5010
E000823_24.3.periodic.trimmo.60.um.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E001958_24.1.periodic.trimmo.60.um.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E003188_24.2.periodic.trimmo.60.um.1,13.190054,9.594140,7.037151,-1.511945,-1.511945,0.341006,-3.875185,-2.894611,-1.540209,-1.380512,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E003953_23.8.periodic.trimmo.60.um.1,7.439115,0.753134,-0.477397,-1.666644,-1.666644,1.566762,-0.320621,-0.048951,-1.828817,-1.666644,...,17.324695,-0.671755,0.546700,-0.261178,-0.945909,4.497051,-0.330914,-0.677626,-0.943841,-0.671755
E004628_24.5.periodic.trimmo.60.um.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E004709_24.7.periodic.trimmo.60.um.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E004898_23.8.periodic.trimmo.60.um.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.089866,3.123426,0.756864,3.349882,1.508875,-0.155927,-1.049277,-1.191721,-1.741592,-1.239181
E005786_23.9.periodic.trimmo.60.um.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E006091_23.5.periodic.trimmo.60.um.1,6.738676,5.471900,12.684754,-1.674267,-1.674267,-0.486124,-1.753948,-3.346891,-1.834991,-1.674267,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
E006493_24.1.periodic.trimmo.60.um.1,8.586070,7.635788,3.293441,-1.441047,-1.441047,-0.229084,-3.191998,-2.026903,-1.585262,-1.441047,...,-0.870998,-0.790442,-0.647449,-0.147259,-1.004367,-0.595347,0.922690,-0.929073,-1.002884,-0.790442


In [271]:
## let's get a list of all the reference genomes present in my DIABIMMUNE analysis,
## based on the column names.
DIABIMMUNE_deletion_cols = [str(col) for col in DIABIMMUNE_deletion_df.columns]
DIABIMMUNE_deletion_reference_genomes = set([x.split(':')[0] for x in DIABIMMUNE_deletion_cols])

DIABIMMUNE_structvar_cols = [str(col) for col in DIABIMMUNE_structvar_df.columns]
DIABIMMUNE_structvar_reference_genomes = set([x.split(':')[0] for x in DIABIMMUNE_structvar_cols])

## now take the union of the two sets. There are 38 reference genomes represented here.
DIABIMMUNE_ref_genomes = DIABIMMUNE_structvar_reference_genomes.union(DIABIMMUNE_deletion_reference_genomes)
num_DIABIMMUNE_ref_genomes = len(DIABIMMUNE_ref_genomes)

## now take the intersection of the two sets. There are 38 reference genomes represented here.
DIABIMMUNE_ref_intersection = DIABIMMUNE_structvar_reference_genomes.intersection(DIABIMMUNE_deletion_reference_genomes)
num_DIABIMMUNE_ref_intersection = len(DIABIMMUNE_ref_intersection)

## the union and intersection are the same!
print(DIABIMMUNE_ref_intersection == DIABIMMUNE_ref_genomes)

True


In [272]:
## now, let's divide the SGVFinder results by case/control cohort samples.
case_deletion_df = DIABIMMUNE_deletion_df[DIABIMMUNE_deletion_df.index.str.contains(case_sample_string)]
control_deletion_df = DIABIMMUNE_deletion_df[DIABIMMUNE_deletion_df.index.str.contains(control_sample_string)]

case_structvar_df = DIABIMMUNE_structvar_df[DIABIMMUNE_structvar_df.index.str.contains(case_sample_string)]
control_structvar_df = DIABIMMUNE_structvar_df[DIABIMMUNE_structvar_df.index.str.contains(control_sample_string)]

In [273]:
def get_refgenome_from_column(col):
    return col.split(":")[0]


def get_genomebins_from_column(col):
    """ this returns a list of all stretches in the cluster"""
    genomebin_list = []
    genomebins = col.split(":")[1].split(";")
    return genomebins


def get_genes_from_column(col, reference_annotations_df, binlen=1000):
    refgenome = get_refgenome_from_column(col)
    genomebins = get_genomebins_from_column(col)
    ## get all gene annotations in the reference genome.
    genome_annotations_df = reference_annotations_df.loc[refgenome]
    ## now get all gene annotations that overlap genomebin_list.
    relevant_annotations_list = []
    for gbin in genomebins:
        startstr, endstr = gbin.split("_")
        int_start = int(startstr) * binlen
        int_end = int(endstr) * binlen
        relevant_annotations = genome_annotations_df[genome_annotations_df.end_pos >= int_start]
        relevant_annotations = relevant_annotations[relevant_annotations.start_pos < int_end]
        relevant_annotations_list.append(relevant_annotations)
        ## concatenate and remove any duplicates.
    all_relevant_annotations = pd.concat(relevant_annotations_list).drop_duplicates()
    return all_relevant_annotations


def filter_column_by_regex(col, reference_annotations_df, regex):
    annotation_df = get_genes_from_column(col, reference_annotations_df)
    annotation_strings = [x for x in list(cur_annotation_df.gene_product) if len(x)]
    if len(list(filter(ARG_regex.search, annotation_strings))):
        regex_matched = True
    else:
        regex_matched = False
    return regex_matched

In [308]:
def getSGVannotations(df, reference_annotations_df, keep_positive=True, threshold=0.0):
    """
    keep_positive is True if examining increase in copy number/presence of gene.
    the threshold is +/- standard deviations from the mean.
    """

    relevant_annotation_string_list = []
    weights = []

    for col in df:
        if keep_positive:
            weight = sum([x for x in df[col] if x > threshold])
        else:
            weight = sum([x for x in df[col] if x < threshold])

        cur_annotation_df = get_genes_from_column(col, reference_annotations_df)
        cur_annotation_strings = [x for x in list(cur_annotation_df.gene_product) if len(x)]
        relevant_annotation_string_list += cur_annotation_strings
        weights.append(weight)
        
    return (relevant_annotation_string_list, weights)

In [309]:
def getDeletionAnnotations(df, reference_annotations_df, keep_positive=True):
    ## keep_positive is True if examining increase in copy number/presence of gene.

    relevant_annotation_string_list = []
    weights = []

    for col in df:
        if keep_positive:
            weight = sum([x for x in df[col] if x == 1])
        else:
            weight = sum([x for x in df[col] if x == 0])

        cur_annotation_df = get_genes_from_column(col, reference_annotations_df)
        cur_annotation_strings = [x for x in list(cur_annotation_df.gene_product) if len(x)]
        relevant_annotation_string_list += cur_annotation_strings
        weights.append(weight)
        
    return (relevant_annotation_string_list, weights)

In [310]:
""" let's examine the annotations of genes in eight categories: case/control * deleted/SGV * positive/negative"""
positive_case_SGV, positive_case_SGV_weights = getSGVannotations(case_structvar_df, reference_annotations_df)

In [311]:
positive_control_SGV, positive_control_SGV_weights = getSGVannotations(control_structvar_df, reference_annotations_df)

In [312]:
negative_case_SGV, negative_case_SGV_weights = getSGVannotations(case_structvar_df, reference_annotations_df,False)

In [313]:
negative_control_SGV, negative_control_SGV_weights = getSGVannotations(control_structvar_df, reference_annotations_df,False)

In [314]:
positive_case_deletionV, positive_case_deletionV_weights = getDeletionAnnotations(case_deletion_df, reference_annotations_df)

In [315]:
positive_control_deletionV, positive_control_deletionV_weights = getDeletionAnnotations(control_deletion_df, reference_annotations_df)

In [316]:
negative_case_deletionV, negative_case_deletionV_weights = getDeletionAnnotations(case_deletion_df, reference_annotations_df,False)

In [317]:
negative_control_deletionV, negative_control_deletionV_weights = getDeletionAnnotations(control_deletion_df, reference_annotations_df,False)

In [323]:
print(len(positive_case_SGV), sum(positive_case_SGV_weights)/21)
print(len(positive_control_SGV), sum(positive_control_SGV_weights)/19)
print(len(negative_case_SGV))
print(len(negative_control_SGV))
print(len(positive_case_deletionV))
print(len(positive_control_deletionV))
print(len(negative_case_deletionV))
print(len(negative_control_deletionV))

3270 1068.1753416970137
3270 974.1080891602601
3270
3270
10380
10380
10380
10380


In [325]:
positive_case_SGV_ARGs = list(filter(ARG_regex.search, positive_case_SGV))
print(positive_case_SGV_ARGs)

positive_control_SGV_ARGs = list(filter(ARG_regex.search, positive_control_SGV))
print(len(positive_control_SGV_ARGs))

negative_case_SGV_ARGs = list(filter(ARG_regex.search, negative_case_SGV))
print(len(negative_case_SGV_ARGs))

negative_control_SGV_ARGs = list(filter(ARG_regex.search, negative_control_SGV))
print(len(negative_control_SGV_ARGs))

positive_case_deletionV_ARGs = list(filter(ARG_regex.search, positive_case_deletionV))
print(len(positive_case_deletionV_ARGs))

positive_control_deletionV_ARGs = list(filter(ARG_regex.search, positive_control_deletionV))
print(len(positive_control_deletionV_ARGs))

negative_case_deletionV_ARGs = list(filter(ARG_regex.search, negative_case_deletionV))
print(len(negative_case_deletionV_ARGs))

negative_control_deletionV_ARGs = list(filter(ARG_regex.search, negative_control_deletionV))
print(len(negative_control_deletionV_ARGs))

['macrolide ABC transporter ATP-binding protein', 'multidrug transporter', 'multidrug resistance protein, MATE family, drug/sodium antiporter', 'tetracycline resistance protein tetQ', 'metallo-beta-lactamase domain protein', 'metallo-beta-lactamase domain protein', 'metallo-beta-lactamase domain protein', 'ABC-type multidrug transport system, ATPase component', 'ABC-type multidrug transport system, ATPase component', 'ABC-type multidrug transport system, ATPase component', 'putative transmembrane MatE family multidrug efflux transporter', 'Streptomycin adenylyltransferase.', 'Beta-lactamase class C and other penicillin binding proteins', 'ABC-type multidrug transport system, ATPase component', 'ABC-type multidrug transport system, ATPase component', 'multiple antibiotic resistance (MarC)-related protein', 'putative 5-nitroimidazole antibiotic resistance protein NimB', 'ABC-type multidrug transport system, ATPase and permease components', 'ABC-type multidrug transport system, ATPase and

## idea for the initial analysis:

### key question: is there a difference in enrichment of amplified ARGs between the samples in the antibiotics+ cohort and the antibiotics- cohort?

Divide samples into the ARG+ and ARG- groups.
For each sample (row):
    For each SGV (column):  
        look up the annotation for the genes in the bin.  
        If it contains an ARG that is amplified (SGV) or present (in the deletion variants)  
            then add '1' to the top row of the contingency table.  
        else,  
            add '1' to the bottom row of the contingency table.  
            
        If the sample is in the ARG+ group, add the entry to the left column, otherwise add it to the right column.
        
        if the SGV is amplified, or if it is present (in the deletion variants)
            